In [7]:
# Install and import required packages
#!pip3 install esda
#!pip install splot
import pandas as pd
import numpy as np
import geopandas as gpd
import libpysal as lps
import esda
from esda.moran import Moran_Local
from esda.moran import Moran
import numpy as np
from splot.esda import plot_moran, moran_scatterplot, lisa_cluster
import matplotlib.pyplot as plt
import seaborn as sns

In [28]:
# Load data
df_2021 = pd.read_csv('data/df_amenity_2021.csv')
df_2021 = df_2021.drop(columns=['Unnamed: 0'], axis=1)
df_bbox = gpd.read_file('data/df_bbox_2017.geojson')

In [29]:
# Combine df_bbox and df_2021
df_2021 = pd.concat([df_bbox, df_2021], axis=1)
df = df_2021.drop(['row_num','col_num','geometry'], axis=1)
df_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.center_lon, df.center_lat), crs='EPSG:4269')

# Read Dallas County shape file
df_geo = gpd.read_file('data/dallas_shape/tl_2021_48_tract.shp')
df_geo.head(2)

# Choose only Dallas county (census tract starting with 48113)
df_geo = df_geo[df_geo['GEOID'].str.startswith('48113')]

In [31]:
# Add geometry info of census tracts to df
df_geo = df_geo[['GEOID','geometry']]
df = gpd.sjoin(df_geo, df_gdf, how='left', op='contains')

# Create a count dataframe
df_count = df.drop(['geometry','index_right','center_lat','center_lon','census_tract'], axis=1)
df_count = df_count.groupby('GEOID').sum()
df_count.reset_index(inplace=True)

# Merge df_count and df (geoid and geometry info)
temp = df[['GEOID','geometry']].drop_duplicates()
df_final = pd.merge(df_count, temp, how='left', on='GEOID')

# Set df_final as geopandas
df_final = gpd.GeoDataFrame(df_final)

/Users/wooyongjung/WJ_Projects/Eviction_Study_2023/.conda/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3493: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [32]:
df_final.head(2)

,GEOID,bar,biergarten,cafe,fast_food,food_court,ice_cream,pub,restaurant,college,...,lounger,marketplace,monastery,photo_booth,place_of_mourning,place_of_worship,public_bath,refugee_site,vending_machine,geometry
0,48113000100,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,"POLYGON ((-96.75301 32.81567, -96.75299 32.815..."
1,48113000201,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,"POLYGON ((-96.76156 32.82989, -96.76154 32.830..."


In [41]:
# Load PLACES data
df_place = pd.read_csv('data/PLACES_2023.csv')

# Select needed columns
needed = ['LocationName', 'Data_Value', 'TotalPopulation', 'Short_Question_Text']
df_place = df_place[needed]

# Create df_obesity
df_obesity = df_place[df_place['Short_Question_Text'] == 'Obesity']
df_obesity = df_obesity.drop(columns=['Short_Question_Text'], axis=1)

In [69]:
df_place['Short_Question_Text'].unique()

array(['COPD', 'Cholesterol Screening', 'Depression',
       'Self-care Disability', 'Obesity', 'Current Smoking',
       'Cancer (except skin)', 'Cervical Cancer Screening',
       'Hearing Disability', 'Colorectal Cancer Screening',
       'Binge Drinking', 'Any Disability', 'Diabetes',
       'Cognitive Disability', 'Current Asthma', 'Health Insurance',
       'Stroke', 'All Teeth Lost', 'Sleep <7 hours',
       'Physical Inactivity', 'Arthritis', 'High Cholesterol',
       'General Health', 'Physical Health',
       'Core preventive services for older men', 'Vision Disability',
       'Mental Health', 'Independent Living Disability',
       'Mobility Disability', 'High Blood Pressure', 'Mammography',
       'Annual Checkup', 'Dental Visit', 'Chronic Kidney Disease',
       'Core preventive services for older women', 'Taking BP Medication',
       'Coronary Heart Disease'], dtype=object)

In [44]:
df_place.groupby('Short_Question_Text').count()

,LocationName,Data_Value,TotalPopulation
Short_Question_Text,,,
All Teeth Lost,527,527,527
Annual Checkup,527,527,527
Any Disability,527,527,527
Arthritis,527,527,527
Binge Drinking,527,527,527
COPD,527,527,527
Cancer (except skin),527,527,527
Cervical Cancer Screening,527,527,527
Cholesterol Screening,527,527,527


In [42]:
df_obesity.head()

,LocationName,Data_Value,TotalPopulation
4,48113006200,41.2,5940
68,48113010601,39.5,5729
111,48113000401,36.4,4830
163,48113002200,28.0,2279
172,48113004400,30.8,3107


In [43]:
print(df_obesity.shape)
print(df_final.shape)

(527, 3)
(645, 121)


In [48]:
# Convert LocationName to string
df_obesity['LocationName'] = df_obesity['LocationName'].apply(str)

In [49]:
# Merge df_final and df_obesity
df_final = pd.merge(df_final, df_obesity, left_on='GEOID', right_on='LocationName', how='left')

In [50]:
df_final.head()

,LocationName,Data_Value,TotalPopulation,GEOID,bar,biergarten,cafe,fast_food,food_court,ice_cream,...,lounger,marketplace,monastery,photo_booth,place_of_mourning,place_of_worship,public_bath,refugee_site,vending_machine,geometry
0,48113006200,41.2,5940,48113006200,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,"POLYGON ((-96.84015 32.73197, -96.84005 32.732..."
1,48113010601,39.5,5729,48113010601,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,"POLYGON ((-96.92630 32.77888, -96.92627 32.778..."
2,48113000401,36.4,4830,48113000401,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON ((-96.84234 32.81615, -96.84222 32.816..."
3,48113002200,28.0,2279,48113002200,0.0,0.0,1.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,"POLYGON ((-96.79236 32.78813, -96.79233 32.788..."
4,48113004400,30.8,3107,48113004400,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,"POLYGON ((-96.85680 32.75602, -96.85679 32.757..."


In [56]:
df_final = gpd.GeoDataFrame(df_final, geometry=df_final.geometry, crs='EPSG:4269')

In [58]:
type(df_final)

geopandas.geodataframe.GeoDataFrame

In [63]:
df_final.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 527 entries, 0 to 526
Columns: 124 entries, LocationName to geometry
dtypes: float64(120), geometry(1), int64(1), object(2)
memory usage: 510.7+ KB


In [67]:
!pip install shap

  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.1/453.1 kB 3.3 MB/s eta 0:00:0000:0100:01
Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)


In [68]:
import shap

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [65]:
lps.weights.Queen.from_dataframe(df_final)

/var/folders/y6/z9h695vx2w3gbjh7z32h4j9m0000gn/T/ipykernel_96538/2307433455.py:1: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  lps.weights.Queen.from_dataframe(df_final)


AttributeError: 'NoneType' object has no attribute 'geom_type'

In [52]:
# Calculate Global Moran's I
w = lps.weights.Queen.from_dataframe(df_final)
w.transform = 'r'

y = df_final['Data_Value']
moran = esda.Moran(y, w)
print(f"Moran's I: {moran.I}, p-value: {moran.p_sim}")

# Calculate local Moran's I
m_local = Moran_Local(y, w)

# Add quadrant (1:HH, 2:LH, 3:LL, 4:HL)
df_final['moran_q'] = m_local.q
df_final['m_local_p'] = m_local.p_sim

/var/folders/y6/z9h695vx2w3gbjh7z32h4j9m0000gn/T/ipykernel_96538/359762132.py:2: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = lps.weights.Queen.from_dataframe(df_final)


AttributeError: 'NoneType' object has no attribute 'geom_type'